# Tactical Analysis & Team Clustering
Analyze team playing styles and cluster teams based on tactical metrics.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go

DATA_DIR = "./data"

print("⚽ Tactical Analysis Pipeline")
print("="*60)

## 1. Load Team Features

In [ ]:
print("\n📂 Loading team features...")
team_features = pd.read_csv(os.path.join(DATA_DIR, "team_features_2024.csv"))
team_features['date'] = pd.to_datetime(team_features['date'])

print(f"✓ Team features loaded: {len(team_features)} records")
print(f"  Teams: {team_features['team_name'].nunique()}")
print(f"  Matchweeks: {team_features['matchweek'].max()}")

## 2. Create Team Style Vectors
Aggregate season-level tactical metrics for each team.

In [ ]:
print("\n" + "="*60)
print("CREATING TEAM STYLE VECTORS")
print("="*60)

# Calculate season averages for tactical features
print("\n  Aggregating tactical metrics...")

tactical_features = [
    'xG', 'xGA', 'shots', 'shots_against',
    'shots_on_target', 'deep', 'ppda', 
    'shot_quality', 'pressing_intensity'
]

team_styles = team_features.groupby('team_name')[tactical_features].mean().reset_index()

# Add form metrics
team_styles['avg_goals_for'] = team_features.groupby('team_name')['goals_for'].mean().values
team_styles['avg_goals_against'] = team_features.groupby('team_name')['goals_against'].mean().values
team_styles['ppg'] = team_features.groupby('team_name')['points'].mean().values
team_styles['win_rate'] = (team_features.groupby('team_name')['result'].apply(lambda x: (x == 'W').mean())).values

print(f"✓ Team style vectors created: {len(team_styles)} teams")
print(f"  Features: {len(tactical_features)} tactical metrics")

## 3. Cluster Teams by Playing Style
Use K-means clustering to group teams with similar tactics.

In [ ]:
print("\n" + "="*60)
print("CLUSTERING TEAMS")
print("="*60)

# Prepare features for clustering
print("\n  Standardizing features...")
feature_cols = tactical_features
X = team_styles[feature_cols].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-means clustering
print("\n  Running K-means clustering (k=5)...")
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
team_styles['cluster'] = kmeans.fit_predict(X_scaled)

print(f"✓ Teams clustered into {n_clusters} groups")

# Show cluster distribution
for cluster_id in range(n_clusters):
    cluster_teams = team_styles[team_styles['cluster'] == cluster_id]['team_name'].tolist()
    print(f"\n  Cluster {cluster_id}: {len(cluster_teams)} teams")
    print(f"    {', '.join(cluster_teams)}")

## 4. Analyze Cluster Characteristics
Describe the tactical profile of each cluster.

In [ ]:
print("\n" + "="*60)
print("CLUSTER CHARACTERISTICS")
print("="*60)

# Calculate cluster centroids
cluster_profiles = team_styles.groupby('cluster')[tactical_features + ['ppg', 'win_rate']].mean()

# Define cluster labels based on characteristics
cluster_labels = {}
for cluster_id in range(n_clusters):
    profile = cluster_profiles.loc[cluster_id]
    
    # Categorize based on key metrics
    if profile['xG'] > cluster_profiles['xG'].mean() and profile['pressing_intensity'] > cluster_profiles['pressing_intensity'].mean():
        label = "Attacking Pressing"
    elif profile['xG'] > cluster_profiles['xG'].mean():
        label = "Attacking Possession"
    elif profile['xGA'] < cluster_profiles['xGA'].mean() and profile['ppda'] < cluster_profiles['ppda'].mean():
        label = "Defensive Deep Block"
    elif profile['pressing_intensity'] > cluster_profiles['pressing_intensity'].mean():
        label = "High Pressing"
    else:
        label = "Balanced"
    
    cluster_labels[cluster_id] = label

team_styles['cluster_label'] = team_styles['cluster'].map(cluster_labels)

print("\n📊 Cluster Profiles:")
for cluster_id in range(n_clusters):
    profile = cluster_profiles.loc[cluster_id]
    label = cluster_labels[cluster_id]
    teams_in_cluster = team_styles[team_styles['cluster'] == cluster_id]
    
    print(f"\n  🎯 Cluster {cluster_id}: {label}")
    print(f"     Teams: {len(teams_in_cluster)}")
    print(f"     Avg xG: {profile['xG']:.2f}")
    print(f"     Avg xGA: {profile['xGA']:.2f}")
    print(f"     Pressing: {profile['pressing_intensity']:.2f}")
    print(f"     Shot Quality: {profile['shot_quality']:.2f}")
    print(f"     PPG: {profile['ppg']:.2f}")

## 5. Visualize Clusters with PCA
Reduce dimensions and visualize team clusters.

In [ ]:
print("\n" + "="*60)
print("PCA VISUALIZATION")
print("="*60)

# Apply PCA
print("\n  Applying PCA (2 components)...")
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

team_styles['pca1'] = X_pca[:, 0]
team_styles['pca2'] = X_pca[:, 1]

print(f"✓ PCA complete")
print(f"  Explained variance: {pca.explained_variance_ratio_.sum():.2%}")

# Create interactive scatter plot
fig = px.scatter(
    team_styles,
    x='pca1',
    y='pca2',
    color='cluster_label',
    hover_name='team_name',
    hover_data={
        'pca1': False,
        'pca2': False,
        'cluster_label': True,
        'xG': ':.2f',
        'xGA': ':.2f',
        'ppg': ':.2f',
        'pressing_intensity': ':.2f'
    },
    title='Premier League Team Tactical Clusters (2024/25)',
    labels={'pca1': 'PC1', 'pca2': 'PC2', 'cluster_label': 'Tactical Style'},
    width=900,
    height=600
)

fig.update_traces(marker=dict(size=15, line=dict(width=1, color='white')))
fig.show()

print("\n📊 Interactive visualization displayed above")

## 6. Team Comparison Radar Charts
Create radar charts to compare team tactical profiles.

In [ ]:
print("\n" + "="*60)
print("TEAM COMPARISON VISUALIZATION")
print("="*60)

def create_radar_chart(team_name, comparison_teams=None):
    """Create radar chart comparing a team to league average or other teams."""
    
    # Get team data
    team_data = team_styles[team_styles['team_name'] == team_name].iloc[0]
    
    # Calculate percentile ranks
    radar_metrics = ['xG', 'xGA', 'shots', 'pressing_intensity', 'shot_quality', 'deep']
    percentiles = []
    
    for metric in radar_metrics:
        # Lower is better for xGA
        if metric == 'xGA':
            percentile = (team_styles[metric] >= team_data[metric]).mean() * 100
        else:
            percentile = (team_styles[metric] <= team_data[metric]).mean() * 100
        percentiles.append(percentile)
    
    # Create figure
    fig = go.Figure()
    
    # Add team trace
    fig.add_trace(go.Scatterpolar(
        r=percentiles,
        theta=radar_metrics,
        fill='toself',
        name=team_name,
        line=dict(color='#00ff87', width=2)
    ))
    
    # Add comparison teams
    if comparison_teams:
        for comp_team in comparison_teams:
            comp_data = team_styles[team_styles['team_name'] == comp_team].iloc[0]
            comp_percentiles = []
            
            for metric in radar_metrics:
                if metric == 'xGA':
                    percentile = (team_styles[metric] >= comp_data[metric]).mean() * 100
                else:
                    percentile = (team_styles[metric] <= comp_data[metric]).mean() * 100
                comp_percentiles.append(percentile)
            
            fig.add_trace(go.Scatterpolar(
                r=comp_percentiles,
                theta=radar_metrics,
                fill='toself',
                name=comp_team,
                opacity=0.6
            ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 100])
        ),
        showlegend=True,
        title=f'Tactical Profile: {team_name} (Percentile Ranks)',
        width=700,
        height=500
    )
    
    return fig

# Example: Compare top teams
print("\n  Creating radar charts for top teams...")
top_teams = team_styles.nlargest(3, 'ppg')['team_name'].tolist()

for i, team in enumerate(top_teams[:2]):  # Show first 2
    fig = create_radar_chart(team, comparison_teams=[top_teams[(i+1) % len(top_teams)]])
    fig.show()
    print(f"✓ Radar chart for {team}")

## 7. Save Results

In [ ]:
print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

# Save team styles with clusters
team_styles.to_csv(os.path.join(DATA_DIR, "team_tactical_styles_2024.csv"), index=False)
print(f"💾 Saved: team_tactical_styles_2024.csv")

# Save cluster profiles
cluster_profiles.reset_index().to_csv(os.path.join(DATA_DIR, "cluster_profiles_2024.csv"), index=False)
print(f"💾 Saved: cluster_profiles_2024.csv")

print("\n" + "="*60)
print("✅ TACTICAL ANALYSIS COMPLETE!")
print("="*60)

print("\n📊 Results Summary:")
print(f"  {n_clusters} tactical clusters identified")
print(f"  {len(team_styles)} teams analyzed")
print(f"  Cluster labels: {list(cluster_labels.values())}")